In [1]:
import bia_bob

In [2]:
import os

# Specify the directory containing Java files
directory_path = r"C:\structure\code\clesperantoj_prototype\src\main\java\net\clesperanto\kernels"
clij3_file_path = "C:/structure/code/clij3/src/main/java/net/clesperanto/CLIJ3.java"

min_tier = 1
max_tier = 8



In [3]:
line = "public static ArrayJ absolute(DeviceJ device, ArrayJ input, ArrayJ output)"

def get_function_name(line):
    return line.split("(")[0].split(" ")[-1]
get_function_name(line)

'absolute'

In [4]:
def get_return_type(line):
    return " ".join(line.split("(")[0].split(" ")[:-1])
get_return_type(line)

'public static ArrayJ'

In [5]:
def get_parameters(line):
    return [f.strip() for f in line.split("(")[1].split(")")[0].split(",")]

get_parameters(line)

['DeviceJ device', 'ArrayJ input', 'ArrayJ output']

In [6]:
def make_java_types(list_of_parameters):
    definitions = []
    calls = []
    for p in list_of_parameters:
        definition_type = p.split(" ")[0]
        parameter_name = p.split(" ")[1]
        if definition_type == "ArrayJ":
            definitions.append("Object " + parameter_name)
            calls.append("push(" + parameter_name + ")")
        else:
            definitions.append(definition_type + " " + parameter_name)
            calls.append(parameter_name)

    return definitions, calls

make_java_types(get_parameters(line))

(['DeviceJ device', 'Object input', 'Object output'],
 ['device', 'push(input)', 'push(output)'])

In [7]:
def function_wrapper(line, tier):
    function_name = get_function_name(line)
    return_type = get_return_type(line).replace("static ", "")
    parameters = get_parameters(line)

    all_but_first_parameters = parameters[1:]

    param_definitions, param_values = make_java_types(all_but_first_parameters)

    param_definitions = ", ".join(param_definitions)
    param_values = ", ".join(param_values)
    
    return f"""
    {return_type} {function_name} ({param_definitions}) {{
        return Tier{tier}.{function_name}(device, {param_values});
    }}
"""

print(function_wrapper(line, 1))


    public ArrayJ absolute (Object input, Object output) {
        return Tier1.absolute(device, push(input), push(output));
    }



In [8]:
output = ""
for tier in range(min_tier, max_tier + 1):
    file_path = os.path.join(directory_path, f"Tier{tier}.java")
    with open(file_path, 'r') as file:
        content = file.read()

        # List to store lines starting with "public static "
        matching_lines = []
        
        # Process each line
        for line in content.splitlines():
            stripped_line = line.lstrip()  # Remove leading spaces
            if stripped_line.startswith("public static "):
                matching_lines.append(stripped_line)

        for line in matching_lines:
            output = output + function_wrapper(line, tier)

In [9]:
print(clij3_file_path)

# Read the content of the file
with open(clij3_file_path, 'r') as file:
    content = file.readlines()


C:/structure/code/clij3/src/main/java/net/clesperanto/CLIJ3.java


In [10]:
# Find the start and end indices of the block
start_index = None
end_index = None
start_marker = "/* BEGIN AUTO-GENERATED FUNCTIONS */"
end_marker = "/* END AUTO-GENERATED FUNCTIONS */"

for i, line in enumerate(content):
    if start_marker in line:
        start_index = i
    elif end_marker in line:
        end_index = i
        break

print (start_index, end_index)

# Replace the block
if start_index is not None and end_index is not None:
    updated_content = content[:start_index + 1] + [output] + content[end_index:]
    
    # Write the modified content back to the file
    with open(clij3_file_path, 'w') as file:
        file.writelines(updated_content)

147 1056
